# A Simple Chatbot Using TF / Keras

This chatbot is implemented using TF/Keras and the training text is kept in a normal text file as a text array. The model will be trained using this text and will be validated against a sample text to check its accuracy

# All common imports

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Bidirectional,GlobalAveragePooling1D,Flatten
import json
from sklearn.preprocessing import LabelEncoder

# Loading few sample chat transcripts from chat-intents.json file stored under ./tmp/simple-chat-bot folder

In [16]:
with open('./tmp/simple-chat-bot/chat-intents.json') as file:
    data = json.load(file)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'], 'responses': ['Hello', 'Hi', 'Hi there']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']}, {'tag': 'wish', 'patterns': ['How are you?', 'HRU', 'How is the day?'], 'responses': ['Fine. Thank You', 'Great. Thanks for asking']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?'], 'responses': ["I'm Agasthya, your bot assistant", "I'm Agasthya, an Artificial Intelligent bot"]}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Agasthya.', 'Just call me as Agasthya']}, {'tag': 'help', 'patterns': ['Could you help me?', 

# Data Pre-Processing

The variable 'training_sentences' holds all the training data (which are the sample messages in each intent category) and the 'training_labels' variable holds all the target labels correspond to each training data.
Then we use 'LabelEncoder()' function provided by scikit-learn to convert the target labels into a model understandable form.

In [17]:
training_sentences = []
training_labels = []
labels = []
responses = []
for intent in data['intents']:
    for patterns in intent['patterns']:
        training_sentences.append(patterns)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
encoder= LabelEncoder()
encoder.fit(training_labels)
training_labels = encoder.transform(training_labels)

# Tokenize the above text

In [18]:
vocab_size = 1000
max_len = 20
oov_token = "<OOV>"

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Convert text tokens to sequences

In [19]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Building & Compiling the model

In [20]:
embedding_dim = 16
num_classes = len(labels)
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(128,return_sequences=True,recurrent_dropout=0.2)))
model.add(Bidirectional(LSTM(128,recurrent_dropout=0.2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 16)            16000     
                                                                 
 bidirectional_6 (Bidirecti  (None, 20, 256)           148480    
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 561290 (2.14 MB)
Trainable params: 56129

# Training the model

In [22]:
epochs = 100  # Increase the number of epochs to give the model more time to learn
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/100
2/2 [==============================] - 0s 79ms/step - loss: 2.2456 - accuracy: 0.1750
Epoch 2/100
2/2 [==============================] - 0s 64ms/step - loss: 2.2418 - accuracy: 0.1750
Epoch 3/100
2/2 [==============================] - 0s 69ms/step - loss: 2.2440 - accuracy: 0.1750
Epoch 4/100
2/2 [==============================] - 0s 76ms/step - loss: 2.2432 - accuracy: 0.2000
Epoch 5/100
2/2 [==============================] - 0s 85ms/step - loss: 2.2408 - accuracy: 0.2000
Epoch 6/100
2/2 [==============================] - 0s 82ms/step - loss: 2.2308 - accuracy: 0.1750
Epoch 7/100
2/2 [==============================] - 0s 67ms/step - loss: 2.2160 - accuracy: 0.1750
Epoch 8/100
2/2 [==============================] - 0s 70ms/step - loss: 2.1890 - accuracy: 0.1750
Epoch 9/100
2/2 [==============================] - 0s 85ms/step - loss: 2.1621 - accuracy: 0.1750
Epoch 10/100
2/2 [==============================] - 0s 75ms/step - loss: 2.1704 - accuracy: 0.1750
Epoch 11/100
2/2 [=

# Function to generate text based on the seed input given

In [23]:
while True:
    inp = input()
    if inp.lower() == "exit":
        break
    result = model.predict(tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
    tag = encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
        if i['tag'] == tag:
            print("ChatBot:"  , np.random.choice(i['responses']))

how are you?
1/1 [==============================] - 1s 1s/step
ChatBot: Fine. Thank You
Who are you?
1/1 [==============================] - 0s 16ms/step
ChatBot: I'm Agasthya, your bot assistant
who is your owner
1/1 [==============================] - 0s 18ms/step
ChatBot: I'm Agasthya, your bot assistant
exit
